# PART - 3
-----------
## Model - 5 & 6 (ExtraTreeRegressor & GradientBoostingRegressor with 2024 Sampling Data )

In [0]:
# Import required libraries
from pyspark.sql import functions as F

In [0]:
# Use the delta table already saved
train_df=spark.table("train_df_2024_delta")

In [0]:
# Create a Function for Sampling the data
def stratified_sample(df, frac=0.04, seed=42):
    """
    Stratified sample across multiple categorical dimensions.
    
    Parameters:
        df   : input Spark DataFrame
        frac : fraction of rows to sample from each stratum
        seed : random seed
    
    Returns:
        sample_df : stratified sample DataFrame
    """
    # Create a combined key for stratification
    df = df.withColumn(
        "strata",
        F.concat_ws("_",
            df["pickup_borough"],
            df["dropoff_borough"],
            df["payment_type"].cast("string"),
            df["taxi_colour"],
            df["month"].cast("string"),
            df["day_of_week"].cast("string"),
            df["hour"].cast("string"),
            df["trip_type"].cast("string"),
            df["RatecodeID"].cast("string")

        )
    )
    
    # Build fractions dictionary (one entry per unique stratum)
    fractions = (
        df.select("strata")
          .distinct()
          .withColumn("fraction", F.lit(frac))
    )
    fractions_dict = {row["strata"]: row["fraction"] for row in fractions.collect()}

    # Apply stratified sampling
    sample_df = df.sampleBy("strata", fractions=fractions_dict, seed=seed)

    # Drop helper column
    sample_df = sample_df.drop("strata")

    return sample_df


In [0]:
# Take a sample_df using stratified sample
sample_df = stratified_sample(train_df, frac=0.04, seed=42)

num_rows = sample_df.count()
num_cols = len(sample_df.columns)

print(f"Rows: {num_rows}, Columns: {num_cols}")

Rows: 1027780, Columns: 21


In [0]:
# Convert the dataframe to pandas dataframe
train_df_sample=sample_df.toPandas()

In [0]:
# Create X_train and y_train
X_train = train_df_sample.drop("total_amount",axis=1)
y_train = train_df_sample["total_amount"]

In [0]:
# Load the saved Preprocessor
import joblib
import os

save_path = "/Volumes/workspace/bde/assignment2"

# Ensure directory exists
os.makedirs(save_path, exist_ok=True)

# Load preprocessor
preprocessor = joblib.load(os.path.join(save_path, "preprocessor.joblib"))

print("✅ Model and preprocessor loaded successfully.")


✅ Model and preprocessor loaded successfully.


In [0]:
# Transform the data using the saved preprocessor
X_train_preprocessed= preprocessor.transform(X_train)

In [0]:
# Train the ExtraTreeRegressor model
from sklearn.tree import ExtraTreeRegressor

model_1 = ExtraTreeRegressor(random_state=42)
model_1.fit(X_train_preprocessed, y_train)

ExtraTreeRegressor(random_state=42)

In [0]:
# Load the saved X_test and y_test
import numpy as np
import os

save_path = "/Volumes/workspace/bde/assignment2"

X_test = np.load(os.path.join(save_path, "X_test_processed.npy"))
y_test = np.load(os.path.join(save_path, "y_test.npy"))

print("✅ Reloaded preprocessed test set")


✅ Reloaded preprocessed test set


In [0]:
# Calculate the RMSE on the test set
from sklearn.metrics import root_mean_squared_error
y_pred = model_1.predict(X_test)
rmse = root_mean_squared_error(y_test, y_pred)
print(f"Extra Tree Regressor RMSE: {rmse:.2f}")

Extra Tree Regressor RMSE: 2.35


In [0]:
# Train the GradientBoostingRegressor model
from sklearn.ensemble import GradientBoostingRegressor
model_2 = GradientBoostingRegressor(random_state=42)
model_2.fit(X_train_preprocessed, y_train)

GradientBoostingRegressor(random_state=42)

In [0]:
# Calculate the RMSE on the test set
from sklearn.metrics import root_mean_squared_error
y_pred = model_2.predict(X_test)
rmse = root_mean_squared_error(y_test, y_pred)
print(f"Gradient Boosting Regressor RMSE: {rmse:.2f}")

Gradient Boosting Regressor RMSE: 2.16
